In [3]:
!export CUDA_VISIBLE_DEVICES=0

'export' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC


In [4]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
# os.environ['CUDA_VISIBLE_DEVICES']='0,1'
os.environ['CUDA_LAUNCH_BLOCKING']='1'
import torch
# torch.cuda.set_device(0)
print(torch.cuda.current_device())


0


In [5]:
torch.cuda.is_available()

True

In [6]:
torch.cuda.device_count()

1

In [7]:
torch.cuda.current_device()

0

In [8]:
# cudaSetDevice(0)
#dataset transcript要在dataset資好夾調
#cuda 在basesolver init調
#dataset要在data.py設定

In [9]:
# %pip install soundfile
# %pip install transformers

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
# os.environ['CUDA_VISIBLE_DEVICES']='0,1'
os.environ['CUDA_LAUNCH_BLOCKING']='1'
import torch
# torch.cuda.set_device(0)
print(torch.cuda.current_device())
#! python
# -*- coding: utf-8 -*-
# Author: kun
# @Time: 2019-10-29 20:29

import yaml
import torch
import argparse
import numpy as np
import pandas as pd

dfs = []
alldf =pd.DataFrame()

class Para(object):
    a=1

def force_cudnn_initialization():
    s = 32
    dev = torch.device('cuda:0')
    torch.nn.functional.conv2d(torch.zeros(s, s, s, s, device=dev), torch.zeros(s, s, s, s, device=dev))
    
#force_cudnn_initialization()
def main():
    # For reproducibility, comment these may speed up training
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Arguments
#     parser = argparse.ArgumentParser(description='Training E2E asr.')
#     parser.add_argument('--config', type=str, help='Path to experiment config.')
#     parser.add_argument('--name', default=None, type=str, help='Name for logging.')
#     parser.add_argument('--logdir', default='log/', type=str,
#                         help='Logging path.', required=False)
#     parser.add_argument('--ckpdir', default='ckpt/', type=str,
#                         help='Checkpoint path.', required=False)
#     parser.add_argument('--outdir', default='result/', type=str,
#                         help='Decode output path.', required=False)
#     parser.add_argument('--load', default=None, type=str,
#                         help='Load pre-trained model (for training only)', required=False)
#     parser.add_argument('--seed', default=0, type=int,
#                         help='Random seed for reproducable results.', required=False)
#     parser.add_argument('--cudnn-ctc', action='store_true',
#                         help='Switches CTC backend from torch to cudnn')
#     parser.add_argument('--njobs', default=32, type=int,
#                         help='Number of threads for dataloader/decoding.', required=False)
#     parser.add_argument('--cpu', action='store_true', help='Disable GPU training.')
#     parser.add_argument('--no-pin', action='store_true',
#                         help='Disable pin-memory for dataloader')
#     parser.add_argument('--test', action='store_true', help='Test the model.')
#     parser.add_argument('--no-msg', action='store_true', help='Hide all messages.')
#     parser.add_argument('--lm', action='store_true',
#                         help='Option for training RNNLM.')
#     # Following features in development.
#     parser.add_argument('--amp', action='store_true', help='Option to enable AMP.')
#     parser.add_argument('--reserve-gpu', default=0, type=float,
#                         help='Option to reserve GPU ram for training.')
#     parser.add_argument('--jit', action='store_true',
#                         help='Option for enabling jit in pytorch. (feature in development)')
#     ###
#     paras = parser.parse_args()
    paras = Para()
#     paras.config = './config/aishell_asr_example_lstm4atthead1-test.yaml'
#     paras.name = None
#     paras.logdir = 'log/'
#     paras.ckpdir = 'ckpt/'
#     paras.outdir = 'result/'
#     paras.load = None
#     paras.seed = 0
#     paras.cudnn_ctc = False
#     paras.cpu = False
#     paras.no_pin = False
#     paras.test = True
#     paras.no_msg = False
#     paras.lm = False
#     paras.amp = False
#     paras.reserve_gpu = 0
#     paras.jit = False
    setattr(paras, 'config', 'config/cv11Lu_asr_lstm4atthead_allvocab-CDRSOB-test-5-1.yaml')
    setattr(paras, 'name', None)
    setattr(paras, 'logdir', 'log/')
    setattr(paras, 'ckpdir', 'LAS_ckpt')
    setattr(paras, 'outdir', 'result/')
    setattr(paras, 'load', None)
    setattr(paras, 'seed', 0)
    setattr(paras, 'cudnn_ctc', False)
    setattr(paras, 'njobs',0)
    setattr(paras, 'cpu', False)
    setattr(paras, 'no_pin', False)
    setattr(paras, 'test', True)
    setattr(paras, 'no_msg', False)
    setattr(paras, 'lm', False)
    setattr(paras, 'amp', False)
    setattr(paras, 'reserve_gpu', 9)
    setattr(paras, 'jit', False)
    setattr(paras, 'gpu', not paras.cpu)
    setattr(paras, 'pin_memory', not paras.no_pin)
    setattr(paras, 'verbose', not paras.no_msg)
    setattr(paras, 'finetune', False)
    setattr(paras, 'binaryClass', True)
#     force_cudnn_initialization()

    config = yaml.load(open(paras.config, 'r'), Loader=yaml.FullLoader)

    np.random.seed(paras.seed)
    torch.manual_seed(paras.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(paras.seed)

    # Hack to preserve GPU ram just incase OOM later on server
    if paras.gpu and paras.reserve_gpu > 0:
        buff = torch.randn(int(paras.reserve_gpu * 1e9 // 4)).cuda()
        del buff

    if paras.lm:
        # Train RNNLM
        from train_lm import Solver

        mode = 'train'
    else:
        if paras.test:
            # Test ASR
            assert paras.load is None, 'Load option is mutually exclusive to --test'
            from Whisper_CDRSOB_test import Solver

            mode = 'test'
        elif paras.finetune:
            assert paras.load is not None
            from finetune_asr import Solver
            mode = 'train'
        elif paras.binaryClass:
            assert paras.load is not None
            from Whisper_CDRSOB_train import Solver
            mode = 'train'
        else:
            # Train ASR
            from train_asr import Solver

            mode = 'train'

    print("\nUsing {} mode\n".format(mode))
    for times in range (15,16):
        alldf =pd.DataFrame()
        for idx in range(5):
            paras.config = f'config/cv11Lu_asr_lstm4atthead_allvocab-CDRSOB-test-5-{idx+1}.yaml'
            config = yaml.load(open(paras.config, 'r'), Loader=yaml.FullLoader)
            config['core']['ckpt'] = f'LAS_ckpt/cv11Lu_asr_lstm4atthead_allvocab-CDRSOB-5fold-{idx+1}_sd0/whisper-v2-15k-r-final-fix.pth'
            solver = Solver(config, paras, mode)

            solver.load_data()
        #     solver.print_model()
            solver.set_model()
            names, hyps, txts, ans = solver.exec()
            dfs.append(pd.DataFrame(
                {'name': names,
                 'hyps': hyps,
                 'truth': txts,
                 'ifcorrect': ans
                }))
            print(dfs[idx])
            print(names)
            print(hyps)
            print(txts)
            print(ans)
            del solver
        for idx, df in enumerate(dfs):
            alldf = pd.concat([alldf, df])#
        #     df.to_csv(f'CDRSOB_result/whisper-WhisperForAudioClassification-v2-{times}k{idx+1}.csv', index=False)
        # alldf.to_csv(f'CDRSOB_result/whisper-WhisperForAudioClassification-v2-{times}k.csv', index=False) 
#             df.to_csv(f'./CDR_result/5folds{times}k_oldASR_4LSTM{idx+1}.csv', index=False)
#         alldf.to_csv(f'./CDR_result/5folds{times}k_oldASR_4LSTM.csv', index=False) 


0


In [2]:
main()


Using test mode

[INFO] Evaluating result of tr. config @ ../LAS_Mandarin_PyTorch-master/config/cv11Lu_asr_lstm4atthead_allvocab-CDRSOB-5fold-1.yaml
[INFO] Evaluating result of tr. ckpt @ LAS_ckpt/cv11Lu_asr_lstm4atthead_allvocab-CDRSOB-5fold-1_sd0/whisper-v2-15k-r-final-fix.pth


c:\Users\naomi\anaconda3\envs\230917\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at sanchit-gandhi/whisper-large-v2-ft-ls-960h and are newly initialized: ['model.classifier.bias', 'model.classifier.weight', 'model.projector.bias', 'model.projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDatasetCDRSOB as Dataset
[VAGDataset] path: data_process, split: ['CTT5-1-16']
Mozillacv11Dataset CTT5-1-16 found wav data: 17
text len: 17
remove None, then wav data: 16, text len: 16
[VAGDataset] path: data_process, split: ['CTTsegment_remove']
Mozillacv11Dataset CTTsegment_remove found wav data: 99
text len: 99
remove None, then wav data: 85, text len: 85
Setup ASR model and optimizer 
('data_process\\CTT5-1-16\\15CTT.wav',)   0.649317741394043   0.0   False
('data_process\\CTT5-1-16\\16CTT.wav',)   0.5942329168319702   3.0   False
('data_process\\CTT5-1-16\\22CTT.wav',)   0.1799914538860321   0.0   False
('data_process\\CTT5-1-16\\28CTT.wav',)   0.4723232388496399   0.5   False
('data_process\\CTT5-1-16\\34CTT.wav',)   0.5695444345474243   0.0   False
('data_process\\CTT5-1-16\\35CTT.wav',)   0.42199063301086426  

Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at sanchit-gandhi/whisper-large-v2-ft-ls-960h and are newly initialized: ['model.classifier.bias', 'model.classifier.weight', 'model.projector.bias', 'model.projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDatasetCDRSOB as Dataset
[VAGDataset] path: data_process, split: ['CTT5-2-16']
Mozillacv11Dataset CTT5-2-16 found wav data: 18
text len: 18
remove None, then wav data: 18, text len: 18
[VAGDataset] path: data_process, split: ['CTTsegment_remove']
Mozillacv11Dataset CTTsegment_remove found wav data: 99
text len: 99
remove None, then wav data: 85, text len: 85
Setup ASR model and optimizer 
('data_process\\CTT5-2-16\\11CTT.wav',)   0.19945800304412842   9.0   False
('data_process\\CTT5-2-16\\25CTT.wav',)   0.2061527669429779   3.0   False
('data_process\\CTT5-2-16\\2CTT.wav',)   0.10719022154808044   0.0   False
('data_process\\CTT5-2-16\\37CTT.wav',)   0.33260682225227356   1.5   False
('data_process\\CTT5-2-16\\39CTT.wav',)   0.2596842050552368   2.0   False
('data_process\\CTT5-2-16\\45CTT.wav',)   0.1653703153133392

...

Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDatasetCDRSOB as Dataset
[VAGDataset] path: data_process, split: ['CTT5-3-16']
Mozillacv11Dataset CTT5-3-16 found wav data: 17
text len: 17
remove None, then wav data: 17, text len: 17
[VAGDataset] path: data_process, split: ['CTTsegment_remove']
Mozillacv11Dataset CTTsegment_remove found wav data: 99
text len: 99
remove None, then wav data: 85, text len: 85
Setup ASR model and optimizer 
('data_process\\CTT5-3-16\\18CTT.wav',)   0.34024184942245483   1.5   False
('data_process\\CTT5-3-16\\1CTT.wav',)   0.15216681361198425   0.0   False
('data_process\\CTT5-3-16\\20CTT.wav',)   0.3707118332386017   3.0   False
('data_process\\CTT5-3-16\\30CTT.wav',)   0.2468717247247696   0.0   False
('data_process\\CTT5-3-16\\32CTT.wav',)   0.17304672300815582   2.0   False
('data_process\\CTT5-3-16\\38CTT.wav',)   0.2388650923967361

Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at sanchit-gandhi/whisper-large-v2-ft-ls-960h and are newly initialized: ['model.classifier.bias', 'model.classifier.weight', 'model.projector.bias', 'model.projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDatasetCDRSOB as Dataset
[VAGDataset] path: data_process, split: ['CTT5-4-16']
Mozillacv11Dataset CTT5-4-16 found wav data: 17
text len: 17
remove None, then wav data: 17, text len: 17
[VAGDataset] path: data_process, split: ['CTTsegment_remove']
Mozillacv11Dataset CTTsegment_remove found wav data: 99
text len: 99
remove None, then wav data: 85, text len: 85
Setup ASR model and optimizer 
('data_process\\CTT5-4-16\\100CTT.wav',)   0.30691954493522644   0.0   False
('data_process\\CTT5-4-16\\12CTT.wav',)   -0.01416914351284504   1.0   False
('data_process\\CTT5-4-16\\17CTT.wav',)   1.1410621404647827   1.0   False
('data_process\\CTT5-4-16\\21CTT.wav',)   -0.012529360130429268   0.0   False
('data_process\\CTT5-4-16\\29CTT.wav',)   0.1072167381644249   0.0   False
('data_process\\CTT5-4-16\\4CTT.wav',)   0.102520853281

Some weights of WhisperForAudioClassification were not initialized from the model checkpoint at sanchit-gandhi/whisper-large-v2-ft-ls-960h and are newly initialized: ['model.classifier.bias', 'model.classifier.weight', 'model.projector.bias', 'model.projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load data for training/validation, store tokenizer and input/output shape
Prepare dataloader for training/validation
Interface for creating all kinds of dataset
import VAGDatasetCDRSOB as Dataset
[VAGDataset] path: data_process, split: ['CTT5-5-16']
Mozillacv11Dataset CTT5-5-16 found wav data: 17
text len: 17
remove None, then wav data: 17, text len: 17
[VAGDataset] path: data_process, split: ['CTTsegment_remove']
Mozillacv11Dataset CTTsegment_remove found wav data: 99
text len: 99
remove None, then wav data: 85, text len: 85
Setup ASR model and optimizer 
('data_process\\CTT5-5-16\\10CTT.wav',)   0.3477659225463867   1.0   False
('data_process\\CTT5-5-16\\13CTT.wav',)   0.40294989943504333   0.0   False
('data_process\\CTT5-5-16\\14CTT.wav',)   0.07796835154294968   0.0   False
('data_process\\CTT5-5-16\\24CTT.wav',)   -0.14164219796657562   1.5   False
('data_process\\CTT5-5-16\\31CTT.wav',)   0.19030246138572693   1.0   False
('data_process\\CTT5-5-16\\44CTT.wav',)   0.3089657723903

In [3]:
for idx, df in enumerate(dfs):
    alldf = pd.concat([alldf, df])#
    # df.to_csv(f'./CDR_result/5folds12knew_4LSTM{idx+1}.csv', index=False)
alldf.to_csv(f'./CDRSOB_result/whisper-v2-15k-r-final-fix.csv', index=False)    

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
y_true = [0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0] 
y_pred = [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0]
target_names = ['non-patient', 'patient']
print(classification_report(y_true, y_pred, target_names=target_names))

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
print('sensitivity: ',tp / (fn+tp))
print('specificity: ',tn / (fp+tn))


In [ ]:
import numpy as np
from sklearn import metrics
y = np.array([0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0])
pred = np.array([0.0005, 0.1006, 0.8517, 0.9374, 0.9875, 0.2795, 0.0041, 0.0005, 0.0005, 0.2542, 0.0002, 0.0014, 0.0029, 0.9852, 0.0029, 3.6719e-05, 0.9708, 0.0003, 1.2828e-05, 0.9127, 0.0006])
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
metrics.auc(fpr, tpr)

In [ ]:
!python eval.py --file ~/LAS_Mandarin_PyTorch-master/result/mozillacv11_asr_stm4atthead-test_test_output.csv